# RAi's Open Source Autocontouring Library

## Installing and importing various packages

In [ ]:
import sys
sys.version

In [ ]:
try:
    from google.colab import output
    !pip install rai==0.2.0-dev7
    
except ModuleNotFoundError:
    %load_ext autoreload
    %autoreload 2

In [ ]:
import pathlib
from io import BytesIO
import zipfile
import tqdm
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

## The contour model configuration

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

## Download some example HNSCC data

In [ ]:
image_paths, structure_path, data_license_path, data_readme_path, rai_license_path = rai.download_hnscc_example()
structure_path

In [ ]:
sorted_image_paths = rai.sort_dicom_image_paths(paths=image_paths)

# Prune the images used just for this demo (makes final download size smaller)
selected_sorted_image_paths = sorted_image_paths[14:78]

# Use the following to instead use the entire image stack:
# selected_sorted_image_paths = sorted_image_paths

## Use `pydicom` to load the DICOM images

In [ ]:
sorted_image_series = [pydicom.dcmread(path) for path in selected_sorted_image_paths]

## Convert the DICOM files into an image stack for AI inference

In [ ]:
grids, image_stack = rai.sorted_image_series_to_image_stack_hfs(
    cfg=cfg, sorted_image_series=sorted_image_series)

## Load the RAi models

In [ ]:
models = rai.load_models(cfg=cfg)

## Run the inference

In [ ]:
predicted_masks = rai.inference(
    cfg=cfg,
    models=models,
    image_stack=image_stack,
    max_batch_size=10,
)

## Convert the AI model output into contour lines

In [ ]:
predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    cfg=cfg,
    grids=grids, 
    masks=predicted_masks, 
    structure_names=cfg["structures"],
)

## View the results within this notebook

In [ ]:
vmin = 0.22
vmax = 0.35
figsize = (8, 8)

In [ ]:
slice_indices, axis_limits = rai.view_ranges_from_contours_by_structure(predicted_contours_by_structure, buffer=20)

rai.plot_contours_by_structure(
    grids=grids,
    images=image_stack,
    contours_by_structure=predicted_contours_by_structure,
    orientation="transverse",
    slice_indices=slice_indices,
    axis_limits=axis_limits,
    structure_names=cfg["structures"],
    figsize=figsize,
    vmin=vmin,
    vmax=vmax,
)

## Create a DICOM structure file from the results

In [ ]:
created_structure_ds = rai.create_dicom_structure_set(
    cfg=cfg, 
    image_series_headers=sorted_image_series, 
    contours_by_structure=predicted_contours_by_structure
)

In [ ]:
created_structure_path = pathlib.Path("RS-RAi.dcm")

pydicom.dcmwrite(filename=created_structure_path, dataset=created_structure_ds, write_like_original=False)

## Save the created structure file and the original DICOM images to a zip file

In [ ]:
save_to_images_directory = selected_sorted_image_paths + [data_license_path, data_readme_path]
save_to_structure_directory = [rai_license_path, created_structure_path]

with zipfile.ZipFile('results.zip', 'w', compression=zipfile.ZIP_DEFLATED) as results_zip:
    for path in save_to_images_directory:
        results_zip.write(path, f"HNSCC/{path.name}")
        
    for path in save_to_structure_directory:
        results_zip.write(path, f"RAi/{path.name}")

In [ ]:
# Provide the following link to users to view the DICOM files

# https://dicom.radiotherapy.ai/local